In [28]:
import subprocess
import time
import random
import pandas as pd

## Evaluating One Candidate

In [ ]:
def apply_passes(input, output, pass_order):
    # passes_str = " ".join(pass_order)
    cmd = ["riscv64-unknown-elf-gcc", "-I./support/", "-O2"]
    cmd+= pass_order
    cmd+= [input, "-o", output]
    # print(" ".join(cmd))
    subprocess.run(cmd, check=True)

def measure_runtime(binary, count=50):
    total_time = 0.0
    for _ in range(count):
        start = time.time()
        result = subprocess.run([f"./{binary}"], capture_output=True, text=True, check=True)
        if result.stdout.strip() == "Benchmark completed successfully with no errors.":
            pass
        else:
            raise ValueError("Benchmark did not complete successfully.")
        end = time.time()
        total_time += (end - start)
    return total_time / count

def measure_size(binary):
    result = subprocess.run(["size", binary], capture_output=True, text=True)
    lines = result.stdout.strip().split("\n")
    if len(lines) < 2:
        return None
    size_fields = lines[1].split()
    total_size = sum(int(x) for x in size_fields[:4])
    return total_size

def evaluate_candidate(pass_order, input, workdir, mode="runtime"):
    candidate = f"{workdir}/candidate_bin"
    try:
        apply_passes(input, candidate, pass_order)
        if mode == "runtime":
            fitness = measure_runtime(candidate)
        elif mode == "size":
            fitness = measure_size(candidate)
        else:
            raise ValueError("Invalid mode")
        return fitness
    except subprocess.CalledProcessError:
        return float("inf")  # Penalize failure

## Genetic Algorithm Operators

In [24]:

def random_candidate(pass_pool, max_len=5):
    length = random.randint(1, max_len)
    return random.sample(pass_pool, length)



def crossover(parent1, parent2, max_len=5):
    # Pick cut points
    p1_cut = random.randint(1, len(parent1))
    p2_cut = random.randint(1, len(parent2))

    # Combine slices
    child = parent1[:p1_cut] + parent2[p2_cut:]

    # Remove duplicates while preserving order
    seen = set()
    deduped_child = []
    for p in child:
        if p not in seen:
            deduped_child.append(p)
            seen.add(p)

    # Clip to max length
    return deduped_child[:max_len]


def mutate(candidate, pass_pool, mutation_rate=0.1, max_len=5):
    # Start with a copy
    mutated = candidate[:]
    
    # Replace existing passes
    for i in range(len(mutated)):
        if random.random() < mutation_rate:
            # Choose a new pass not in the candidate
            available = list(set(pass_pool) - set(mutated))
            if available:
                mutated[i] = random.choice(available)

    # Random insertion
    if len(mutated) < max_len and random.random() < mutation_rate:
        available = list(set(pass_pool) - set(mutated))
        if available:
            insert_pos = random.randint(0, len(mutated))
            mutated.insert(insert_pos, random.choice(available))

    # Random deletion
    if len(mutated) > 1 and random.random() < mutation_rate:
        del_pos = random.randint(0, len(mutated) - 1)
        mutated.pop(del_pos)

    return mutated



## Running the GA loop

In [ ]:
import pandas as pd

def run_ga(input_bc, pass_pool, generations=10, pop_size=20, seq_length=5):
    population = [random_candidate(pass_pool, seq_length) for _ in range(pop_size)]
    stats = []

    for generation in range(generations):
        print(f"Generation {generation}")
        fitnesses = []
        for candidate in population:
            fitness = evaluate_candidate(candidate, input_bc, "./scratch")
            fitnesses.append((fitness, candidate))
            print(f"Candidate {candidate} => Fitness {fitness}")

        # Selection
        fitnesses.sort(key=lambda x: x[0])
        population = [cand for _, cand in fitnesses[:(pop_size // 2 - 5)]]
        # Add the worst 5 candidates to the population
        population += [cand for _, cand in fitnesses[-5:]]

        # Crossover and mutation
        new_population = []
        while len(new_population) < pop_size:
            parents = random.sample(population, 2)
            c1 = crossover(parents[0], parents[1])
            c1 = mutate(c1, pass_pool)
            new_population.append(c1)

        population = new_population

        # Collect stats for the generation
        best_fitness, best_candidate = fitnesses[0]
        worst_fitness, worst_candidate = fitnesses[-1]
        avg_fitness = sum(f[0] for f in fitnesses) / len(fitnesses)
        stats.append({
            "Generation": generation,
            "Best Candidate": best_candidate,
            "Worst Candidate": worst_candidate,
            "Best Time": best_fitness,
            "Worst Time": worst_fitness,
            "Average Time": avg_fitness
        })

    # Final best
    best = min(fitnesses, key=lambda x: x[0])
    print(f"Best candidate: {best[1]} with fitness {best[0]}")

    # Create a DataFrame from stats
    stats_df = pd.DataFrame(stats)
    # print(stats_df)
    return stats_df


## Example usage

In [36]:
PASS_POOL = [
    "-fgcse-after-reload",
    "-fipa-cp-clone",
    "-floop-interchange",
    "-floop-unroll-and-jam",
    "-fpeel-loops",
    "-fpredictive-commoning",
    "-fsplit-loops",
    "-fsplit-paths",
    "-ftree-loop-distribution",
    "-ftree-partial-pre",
    "-funswitch-loops",
    "-fvect-cost-model=dynamic",
    "-fversion-loops-for-strides",
]

In [46]:
sources = ["src/aha-mont64/mont64.c"]


stats_df = run_ga(sources[0], PASS_POOL, generations=30, pop_size=100, seq_length=5)

Generation 0
Candidate ['-floop-unroll-and-jam', '-fgcse-after-reload', '-fipa-cp-clone'] => Fitness 0.005041813850402832
Candidate ['-fgcse-after-reload', '-floop-unroll-and-jam', '-fversion-loops-for-strides', '-fpeel-loops'] => Fitness 0.004735374450683593
Candidate ['-ftree-partial-pre', '-fversion-loops-for-strides'] => Fitness 0.004602622985839844
Candidate ['-fsplit-paths', '-ftree-loop-distribution', '-floop-unroll-and-jam', '-fipa-cp-clone', '-fsplit-loops'] => Fitness 0.004675173759460449
Candidate ['-fsplit-loops', '-fipa-cp-clone', '-fpeel-loops'] => Fitness 0.004869043827056885
Candidate ['-ftree-partial-pre', '-floop-interchange'] => Fitness 0.0045563578605651855
Candidate ['-ftree-partial-pre', '-fpredictive-commoning', '-floop-interchange', '-fvect-cost-model=dynamic'] => Fitness 0.004576659202575684
Candidate ['-floop-unroll-and-jam', '-floop-interchange', '-fsplit-paths'] => Fitness 0.00471419095993042
Candidate ['-floop-interchange', '-fversion-loops-for-strides', '-

In [44]:
stats_df

,Generation,Best Candidate,Worst Candidate,Best Time,Worst Time,Average Time
0,0,"[-fipa-cp-clone, -fversion-loops-for-strides, ...","[-funswitch-loops, -floop-interchange, -fpredi...",0.004565,0.008533,0.005015
1,1,"[-fversion-loops-for-strides, -fpredictive-com...","[-fsplit-paths, -fgcse-after-reload, -funswitc...",0.004542,0.005142,0.004688
2,2,"[-fipa-cp-clone, -fsplit-loops, -ftree-partial...","[-floop-interchange, -ftree-loop-distribution,...",0.004555,0.005652,0.004698
3,3,"[-fsplit-loops, -funswitch-loops]","[-floop-interchange, -ftree-loop-distribution]",0.004525,0.005428,0.004673
4,4,"[-fsplit-loops, -fpeel-loops, -floop-unroll-an...","[-fsplit-loops, -fpredictive-commoning, -fvers...",0.004487,0.005641,0.004753
5,5,"[-fsplit-paths, -funswitch-loops, -fgcse-after...","[-floop-interchange, -fversion-loops-for-strid...",0.004544,0.005487,0.004897
6,6,"[-fpeel-loops, -ftree-partial-pre, -fsplit-paths]","[-fipa-cp-clone, -funswitch-loops, -fgcse-afte...",0.004507,0.005406,0.004707
7,7,"[-fipa-cp-clone, -fversion-loops-for-strides, ...",[-fsplit-paths],0.004464,0.005566,0.004700
8,8,[-fipa-cp-clone],[-fversion-loops-for-strides],0.004530,0.009367,0.005257
9,9,"[-fipa-cp-clone, -fgcse-after-reload]","[-fpeel-loops, -ftree-loop-distribution, -fpre...",0.004526,0.005697,0.004835


In [47]:
stats_df

,Generation,Best Candidate,Worst Candidate,Best Time,Worst Time,Average Time
0,0,"[-fvect-cost-model=dynamic, -funswitch-loops, ...",[-fipa-cp-clone],0.004478,0.005760,0.004828
1,1,"[-ftree-partial-pre, -fpredictive-commoning, -...","[-fsplit-paths, -fpredictive-commoning]",0.004466,0.005619,0.004603
2,2,"[-fversion-loops-for-strides, -ftree-loop-dist...",[-fipa-cp-clone],0.004494,0.005263,0.004587
3,3,"[-floop-unroll-and-jam, -floop-interchange, -f...","[-fversion-loops-for-strides, -ftree-loop-dist...",0.004505,0.005466,0.004591
4,4,[-fsplit-paths],"[-fvect-cost-model=dynamic, -ftree-loop-distri...",0.004495,0.005355,0.004599
5,5,"[-fversion-loops-for-strides, -ftree-loop-dist...",[-fvect-cost-model=dynamic],0.004482,0.005418,0.004595
6,6,[-fvect-cost-model=dynamic],"[-funswitch-loops, -ftree-partial-pre]",0.004500,0.005673,0.004665
7,7,[-fvect-cost-model=dynamic],"[-floop-unroll-and-jam, -ftree-loop-distributi...",0.004512,0.005101,0.004666
8,8,"[-fvect-cost-model=dynamic, -fipa-cp-clone, -f...","[-floop-unroll-and-jam, -fsplit-paths, -ftree-...",0.004518,0.005535,0.004724
9,9,"[-fvect-cost-model=dynamic, -fsplit-loops, -fp...",[-floop-interchange],0.004549,0.005435,0.004682


In [49]:
stats_df.iloc[2]

Generation                                                         2
Best Candidate     [-fversion-loops-for-strides, -ftree-loop-dist...
Worst Candidate                                     [-fipa-cp-clone]
Best Time                                                   0.004494
Worst Time                                                  0.005263
Average Time                                                0.004587
Name: 2, dtype: object